In [1]:
import pyspark.sql.functions as F
from pyspark.sql.functions import monotonically_increasing_id, lit
from notebookutils import mssparkutils
from synapse.ml.services import AzureSearchWriter

StatementMeta(, 4791253f-2b1f-4109-9f56-d7d1c074b7f6, 3, Finished, Available)

In [2]:
bbcnews = spark.sql(f"SELECT * FROM SilverLakehouse.{YOUR_TABLE_NAME}")

StatementMeta(, 4791253f-2b1f-4109-9f56-d7d1c074b7f6, 4, Finished, Available)

In [3]:
#### Translate Category to French ####
mapping_category = {
    'entertainment' : "divertissement",
    'business' : 'affaire',
    'politics' : "politique",
    'tech' : "technologie",
    'sport' : 'sport'
}

# Create Spark DataFrame
category = spark.sparkContext.parallelize([(k,)+(v,) for k,v in mapping_category.items()]).toDF(['category_en','category_fr'])

bbcnews = bbcnews.join(category, (bbcnews["category"] == category["category_en"]), 'left') \
    .drop(*['category_en'])

StatementMeta(, 4791253f-2b1f-4109-9f56-d7d1c074b7f6, 5, Finished, Available)

In [4]:
bbcnews = bbcnews.withColumn('content_fr', F.format_string("%s \n Tags: \n Catégorie: %s", bbcnews.content_fr, bbcnews.category_fr))
bbcnews = bbcnews.withColumn('content', F.format_string("%s \n Tags: \n Category: %s", bbcnews.content, bbcnews.category))

StatementMeta(, 4791253f-2b1f-4109-9f56-d7d1c074b7f6, 6, Finished, Available)

In [ ]:
# Fetch Search Key from Key Vault
search_key = mssparkutils \
    .credentials \
    .getSecret(f'https://{YOUR_KEY_VAULT}.vault.azure.net/', f'{YOUR_SECRET_NAME}')

In [ ]:
# Search Service Information
search_service_dns = "your_search_service_dns"
search_key = "your_search_service_key"
search_index = "bbcnews"

In [6]:
bbcnews.select(*["filename", "category", "category_fr", "title", "content", "title_fr", "content_fr", "embedding"]) \
    .withColumn("DocID", monotonically_increasing_id().cast("string")) \
    .withColumn("SearchAction", lit("upload")) \
    .writeToAzureSearch(
        subscriptionKey=search_key,
        actionCol="SearchAction",
        serviceName=your_search_service_dns,
        indexName=search_index,
        keyCol="DocID",
    )

StatementMeta(, 4791253f-2b1f-4109-9f56-d7d1c074b7f6, 8, Finished, Available)